In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [2]:
cd /gdrive/MyDrive/fire-detection/

/gdrive/MyDrive/fire-detection


In [3]:
pwd

'/gdrive/MyDrive/fire-detection'

In [4]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
TRAINING_DIR = "datasets/Train"
training_datagen = ImageDataGenerator(rescale = 1./255,
                                  horizontal_flip=True,
                                  rotation_range=30,
                                  height_shift_range=0.2,
                                  fill_mode='nearest')
VALIDATION_DIR = "datasets/Validation"
validation_datagen = ImageDataGenerator(rescale = 1./255)
train_generator = training_datagen.flow_from_directory(TRAINING_DIR,
                                         target_size=(224,224),
                                         class_mode='categorical',
                                         batch_size = 64)
validation_generator = validation_datagen.flow_from_directory(      
                                           VALIDATION_DIR,
                                           target_size=(224,224),
                                           class_mode='categorical',
                                           batch_size= 16)

Found 2780 images belonging to 2 classes.
Found 239 images belonging to 2 classes.


In [5]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(96, (11,11), strides=(4,4), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size = (3,3), strides=(2,2)))
model.add(Conv2D(256, (5,5), activation='relu'))
model.add(MaxPooling2D(pool_size = (3,3), strides=(2,2)))
model.add(Conv2D(384, (5,5), activation='relu'))
model.add(MaxPooling2D(pool_size = (3,3), strides=(2,2)))

model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 54, 54, 96)        34944     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 26, 26, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 256)       614656    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 6, 384)         2457984   
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 2, 384)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1536)              0

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001), metrics=['acc'])

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', patience=50)
mc = ModelCheckpoint('CNN_best_model.h5', monitor='val_loss', mode='min', save_best_only=True)
history = model.fit(train_generator, steps_per_epoch = 15, epochs = 50, validation_data = validation_generator, callbacks=[es, mc])

Epoch 1/50
15/15 [==============================] - 347s 23s/step - loss: 0.6433 - acc: 0.5807 - val_loss: 0.3921 - val_acc: 0.8326
Epoch 2/50
15/15 [==============================] - 235s 16s/step - loss: 0.4269 - acc: 0.8196 - val_loss: 0.3302 - val_acc: 0.8494
Epoch 3/50
15/15 [==============================] - 167s 11s/step - loss: 0.3376 - acc: 0.8646 - val_loss: 0.2420 - val_acc: 0.9247
Epoch 4/50
15/15 [==============================] - 120s 8s/step - loss: 0.2866 - acc: 0.8776 - val_loss: 0.2313 - val_acc: 0.9163
Epoch 5/50
15/15 [==============================] - 103s 7s/step - loss: 0.2604 - acc: 0.8944 - val_loss: 0.2605 - val_acc: 0.9038
Epoch 6/50
15/15 [==============================] - 96s 6s/step - loss: 0.2536 - acc: 0.8894 - val_loss: 0.2504 - val_acc: 0.9121
Epoch 7/50
15/15 [==============================] - 93s 6s/step - loss: 0.2741 - acc: 0.8965 - val_loss: 0.2355 - val_acc: 0.9163
Epoch 8/50
15/15 [==============================] - 90s 6s/step - loss: 0.2404 - a